In [1]:
from selenium import webdriver
from selenium_stealth import stealth # делает похожим на настоящего пользователя
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np

In [59]:
options = webdriver.ChromeOptions()
options.add_argument("start_maximized")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
#chromedriver должен соответствовать актуальной версии chrom, тут используется версия chrom 107.0.5304.121
#версия chromedriver - 107.0.5304.62 (все работает!)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

s_area = [] #область
s_cities = [] #город
s_trc = [] #ТРЦ
s_specifications = [] #характеристики
a = 0

url = f"https://trc.ru/malls" #загружаем сайта trc.ru со всеми регионами
driver.get(url)
    
sleep(1)

#выгружаем всю информацию по классам, а потом заходим в каждый класс и вытаскиваем ссылку области
link_area = driver.find_elements_by_class_name('cs-homepage-category') #выгружаем класс cs-homepage-category
for i in link_area:
    links_area = i.find_element_by_tag_name('a').get_attribute("href") #вытаскиваем ссылку
    s_area.append(links_area) #складываем в словарь 1 раз

#бегу по ссылкам областей, захожу в каждую и вытаскиваю ссылки городов
for j in tqdm(s_area, 'Обработанно ссылок:'):
    driver.get(j)
    sleep(1)
    link_cities = driver.find_elements_by_class_name('cs-homepage-category') #выгружаем класс cs-homepage-category
    for i in link_cities:
        links_cities = i.find_element_by_tag_name('a').get_attribute("href") #вытаскиваем ссылку
        s_cities.append(links_cities) #складываем в словарь 1 раз

#бегу по ссылкам городов, захожу в каждую и вытаскиваю ссылки ТРЦ
for j in tqdm(s_cities, 'Обработанно ссылок:'):
    driver.get(j)
    sleep(1)
    link_trc = driver.find_elements_by_class_name('cs-homepage-category') #выгружаем класс cs-homepage-category
    for i in link_trc:
        links_trc = i.find_element_by_tag_name('a').get_attribute("href") #вытаскиваем ссылку
        s_trc.append(links_trc) #складываем в словарь 1 раз

for j in tqdm(s_trc, 'Обработанно ссылок:'):
    a = a + 1 #счетчик
    s_specifications.clear() #очищаем список значений
    
    driver.get(j)
    sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'lxml') #передаем данный в суп
    
    #Название ТРЦ
    trc_name = soup.find('h1', class_ = 'entry-title').text
    name = ['Название ТРЦ', trc_name]
    s_specifications.append(name)
    
    #Город, адрес и координаты
    address = soup.find('div', class_ = 'entry-content').findAll('ul')
    trc_city = address[0].findAll('li')[1].text.split(" ")[2]
    city = ['Город', trc_city]
    s_specifications.append(city)
    
    trc_address = address[0].findAll('li')[2].text.split(":")[1].strip()
    addr = ['Адрес', trc_address]
    s_specifications.append(addr)
    
    try:
        trc_latitude = address[1].findAll('li')[0].text.split(" ")[1]
        trc_longitude = address[1].findAll('li')[1].text.split(" ")[1]
    except:
        trc_latitude = "-"
        trc_longitude = "-"
    lat = ['Широта', trc_latitude]
    lon = ['Долгота', trc_longitude]
    s_specifications.append(lat)
    s_specifications.append(lon)
    
    #Характеристики
    table = soup.find('table', class_ = 'wp-block-table').find('tbody').findAll('tr')
    k = len(table)
    for i in range(0, k):
        tb = table[i].text.split(':')
        s_specifications.append(tb)
    
    #Ссылка
    lnk = ['Ссылка', j] #создаем список для ссылки
    s_specifications.append(lnk) #складываем в список

    df = pd.DataFrame(s_specifications)
    df = df.set_index(0)    

    if a == 1:
        df_result = df
    if a > 1:
        df_result = pd.concat([df_result, df], axis=1)

#создаем копию и транспонируем
df_result_copy = df_result.copy()
df_result_copy = df_result_copy.transpose()

#Сохраняем в Excel
writer = pd.ExcelWriter('ТРЦ.xlsx')
df_result_copy.to_excel(writer, 'трц')
writer.save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_19628\1150228301.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
C:\Users\skapr\AppData\Local\Temp\ipykernel_19628\1150228301.py:30: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  link_area = driver.find_elements_by_class_name('cs-homepage-category') #выгружаем класс cs-homepage-category
C:\Users\skapr\AppData\Local\Temp\ipykernel_19628\1150228301.py:32: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  links_area = i.find_element_by_tag_name('a').get_attribute("href") #вытаскиваем ссылку
Обработанно ссылок::   0%|                                                                      | 0/63 [00:00<?, ?it/s]C:\Users\skapr\AppData\Local\Temp\ipykernel_19628\1

In [57]:
options = webdriver.ChromeOptions()
options.add_argument("start_maximized")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
#chromedriver должен соответствовать актуальной версии chrom, тут используется версия chrom 107.0.5304.121
#версия chromedriver - 107.0.5304.62 (все работает!)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

url = f"https://trc.ru/malls/moskovskaya-oblast/g-moskva/rivera" #загружаем сайта trc.ru со всеми регионами
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'lxml') #передаем данный в суп

s_specifications = []

table = soup.find('table', class_ = 'wp-block-table').find('tbody').findAll('tr')
k = len(table)
for i in range(0, k):
    tb = table[i].text.split(':')
    s_specifications.append(tb) #складываем в словарь 1 раз


C:\Users\skapr\AppData\Local\Temp\ipykernel_19628\2936771701.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=r"C:\Python\chromedriver.exe")
